In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import struct

import cartopy
from cartopy import crs as ccrs
import matplotlib 
from matplotlib import pyplot as plt
import os
from os.path import join, exists
from os import mkdir
import scipy
import netCDF4
import matplotlib.ticker as mticker
from cartopy.mpl.ticker import (LongitudeFormatter, LatitudeFormatter,
                                LatitudeLocator, LongitudeLocator)
import pandas as pd
import matplotlib.path as mpath
from matplotlib.colors import TwoSlopeNorm 

time_scale=86400/2/np.pi
length_scale=6370*1000
PSI_scale=length_scale**2/time_scale

with open("/scratch/hz1994/blocking/data_MMmodel/filepath.txt","r") as fi:
    for ln in fi:
        if ln.startswith("dimensionalized_filepath"):
            dim_path=ln.strip().split('\t')[1]
        if ln.startswith("TMindex_filepath"):
            TMindex_path=ln.strip().split('\t')[1]  
        if ln.startswith("nondimensionalized_filepath"):
            nondim_path=ln.strip().split('\t')[1]
        if ln.startswith("DGindex_filepath"):
            DGindex_path=ln.strip().split('\t')[1]  
        if ln.startswith("fig_filepath" ):
            fig_path=ln.strip().split('\t')[1]  
print(dim_path)
print(nondim_path)
print(TMindex_path)
print(DGindex_path)
print(fig_path)


/ext3/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/scratch/hz1994/blocking/data_MMmodel/dim/
/scratch/hz1994/blocking/data_MMmodel/nondim/
/scratch/hz1994/blocking/data_MMmodel/TMindex/
/scratch/hz1994/blocking/data_MMmodel/DGindex/
/scratch/hz1994/blocking/data_MMmodel/fig_MMmodel/


# Compute composite of T>=5 and T=0 for MM model

In [ ]:
Z500=xr.open_dataarray(dim_path+"dim_Z500_1250k_lowpass3dys.nc" )
Tk=xr.open_dataarray(DGindex_path+"Atl_Tk_1250k_lowpass3dys.nc")


Z500=Z500.sel(latitude=slice(20,90))
std_z=Z500.std(dim='time')
mean_z=Z500.mean(dim='time')
std_z.to_netcdf(fig_path+"std_z.nc")
mean_z.to_netcdf(fig_path+"mean_z.nc")
blocking_composite=Z500[Tk>=5].mean(dim="time")
non_blocking_composite=Z500[Tk==0].mean(dim="time")
latitudes=Z500.latitude.sel(latitude=slice(20,90))
longitudes=Z500.longitude
latitudes.to_netcdf(fig_path+"latitudes.nc")
longitudes.to_netcdf(fig_path+"longitudes.nc")
blocking_composite.to_netcdf(fig_path+"blocking_composite_MM.nc")
non_blocking_composite.to_netcdf(fig_path+"non_blocking_composite.nc")


# Compute blocking statistics for MMmodel

In [2]:
def IB2duration(lsbyear,day): #5 days duration criteria
    start=lsbyear.copy(deep=True)
    for i in range(day): # identify the points with the forward 1,2,3,4 being true (within LSB). notice we pad the shift by false at the end since a blocking event cannot start there (<5days)
        start=start & lsbyear.shift(time=-i,fill_value=False)
    event=start.copy(deep=True) # include the points forward 1,2,3,4 
    for i in range(day):
        event=event | start.shift(time=i,fill_value=False)
    return event
def fmt(x):
    s = f"{x:.1f}"
    if s.endswith("0"):
        s = f"{x:.0f}"
    return rf"{s} " if plt.rcParams["text.usetex"] else f"{s} "

zprime=xr.open_dataarray(dim_path+"dim_Zprime500_1250k_lowpass3dys.nc" )

M=100
T=5 

zprime=zprime.sel(latitude=slice(20,90))
IB_positive=zprime>=M
blockings_positive=IB2duration(IB_positive,T)
mean=blockings_positive.mean(dim='time')
start=(((blockings_positive.data[1:])*1-(blockings_positive.data[:-1])*1)==1).sum(axis=0)
pltstart=mean.copy(deep=True,data=start)

num_MM=pltstart
percent_MM=mean*100 
num_MM.to_netcdf(fig_path+"num_MM.nc")
percent_MM.to_netcdf(fig_path+"percent_MM.nc")

KeyboardInterrupt: 

# Compute blocking statistics for ERA5

In [7]:
# ds_all=xr.open_dataset("/scratch/hz1994/blocking/DG/1959-2021_data/1959-2021-integrate-ds_new.nc")

ds_all=xr.open_dataset("/scratch/hz1994/blocking/data_era5/1940-1958/era5-1940-2022-z.nc").sel(level=500)

dsanomalies=ds_all['z'].groupby('time.dayofyear')-ds_all['z'].groupby('time.dayofyear').mean()
scale=1/np.sin(dsanomalies.coords['latitude']*np.pi/180)*np.sin(45*np.pi/180)
# zprime=(dsanomalies*scale).sel(latitude=np.arange(20,90+2.5,2.5))
zprime=(dsanomalies*scale) 
startyear=1959
endyear=2021

M=150
T=5

DJF=True
strDJF=DJF*'DJF'
if DJF:
    zprime=zprime.sel(time=slice(str(startyear)+'-12-01',str(endyear)+'-03-01'))
    zprime=zprime.sel(time=zprime.time.dt.month.isin([12,1,2]))
else:
    zprime=zprime.sel(time=slice(str(startyear),str(endyear)))

IB_positive=zprime>=M
IB_negative=zprime<-M
IB_both=IB_positive|IB_negative
blockings_positive=IB2duration(IB_positive,T)
blockings_negative=IB2duration(IB_negative,T)
blockings_both=IB2duration(IB_both,T)

start=(((blockings_positive.data[1:])*1-(blockings_positive.data[:-1])*1)==1).sum(axis=0)
pltstart=blockings_positive.mean(dim='time').copy(deep=True,data=start)

num_era5=pltstart
percent_era5=blockings_positive.mean(dim='time')*100

num_era5.to_netcdf(fig_path+"num_blocking_era5.nc")
percent_era5.to_netcdf(fig_path+"percent_blocking_era5.nc")

In [9]:
ds_all['latitude'].to_netcdf(fig_path+"latitudes.nc")
ds_all['longitude'].to_netcdf(fig_path+"longitudes.nc.nc")


In [10]:
ds_all

<xarray.Dataset>
Dimensions:    (time: 30316, longitude: 90, latitude: 18)
Coordinates:
  * time       (time) datetime64[ns] 1940-01-01 1940-01-02 ... 2022-12-31
  * longitude  (longitude) float64 -180.0 -176.0 -172.0 ... 168.0 172.0 176.0
  * latitude   (latitude) float64 21.29 25.16 29.03 32.9 ... 79.34 83.2 87.04
    level      int32 500
Data variables:
    z          (time, latitude, longitude) float64 ...
Attributes:
    CDI:          Climate Data Interface version 2.0.5 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Wed Jul 03 20:27:42 2024: cdo mergetime era5-1940-1958-z.nc...
    CDO:          Climate Data Operators version 2.0.5 (https://mpimet.mpg.de...